# Extra notebook - Exporting graphs and data for external visualization, further calculations
## Project: Growing Urban Bicycle Networks

This is a temporary notebook which was needed to export data for creating the growbike.net platform. It takes the produced graphs or data from previous steps and exports the data in a format in which it can be used in an external visualization, or for further calculations. 

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-09-25  
Last modified: 2021-06-26

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

In [ ]:
# Run all parameter sets
poi_source_list = ["grid", "railwaystation"]
prune_measure_list = ["betweenness", "closeness", "random"]
combs = list(itertools.product(poi_source_list, prune_measure_list))

## Igraph picklez of simplified carconstrictedbike networks

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Exporting carconstrictedbike to picklez")
    
    # Load existing
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["exports"] + placeid + "/" + placeid + '_carall.picklez', 'wb') as f:
        G_carall_simplified = simplify_ig(G_carall)
        G_carall_simplified.write_picklez(fname = f)
    if debug: map_center = nxdraw(G_carall, "carall")
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    if debug:
        fig = initplot()
        nxdraw(G_carall_simplified, "abstract", map_center, nodesize = 0, weighted = True, maxwidthsquared = 500/100)
        plt.savefig(PATH["exports"] + placeid + "/" + placeid + '_carallweighted.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()
    for GT, prune_quantile in zip(res["GTs"], tqdm(res["prune_quantiles"], desc = "Growth stages", leave = False)):
        if prune_quantile in prune_quantiles: #[0.5,1]:
            GT_carconstrictedbike = copy.deepcopy(G_carall)
            constrict_overlaps(GT_carconstrictedbike, GT)
            GT_carconstrictedbike = simplify_ig(GT_carconstrictedbike)
            if debug:
                fig = initplot()
                nxdraw(GT_carconstrictedbike, "abstract", map_center, nodesize = 0, weighted = True, maxwidthsquared = 500)
                plt.savefig(PATH["exports"] + placeid + "/" + placeid + '_carconstrictedbike_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
                plt.close()
            with open(PATH["exports"] + placeid + "/" + placeid + '_carconstrictedbike_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.picklez', 'wb') as f:
                GT_carconstrictedbike.write_picklez(fname = f)

## Igraph to GeoJSON

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    
    print(placeid + ": Exporting streets to GeoJSON")
    Gs = {}
    for networktype in networktypes:
        Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
        Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
    for nw, G in Gs.items():
        G_geojson = ig_to_geojson(G)
        with open(PATH["exports_json"] + placeid + "/" + placeid + "_" + nw + '.json', 'w') as f:
            geojson.dump(G_geojson, f)
    
    
    print(placeid + ": Exporting simulation results to GeoJSON")
    for poi_source, prune_measure in combs:
        # Load results
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure
        resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
        res = pickle.load(resultfile)
        resultfile.close()
        if debug: pp.pprint(res)

        for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
            GT_geojson = ig_to_geojson(GT)
            with open(PATH["exports_json"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.json', 'w') as f:
                geojson.dump(GT_geojson, f)

## Railwaystation POIs gdf to GeoJSON

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    try:
        poi_source_here = "railwaystation"
        poi_gdf = gpd.GeoDataFrame.from_file(PATH["data"] + placeid + "/" + placeid + '_' + 'poi_' + poi_source_here + '.gpkg')
    #         pp.pprint(poi_gdf.geometry.x)
        with open(PATH["exports_json"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '.json', 'w') as f:
            geojson.dump(gdf_to_geojson(poi_gdf, poi_gdf.keys()), f)
    except:
        print(placeid + " did not work.")

## Grid and railwaystation POIs to lat/lon list

Given [placeid]_poi_grid_nnidscarall, fetch and export x,y (lat,lon) from [placeid]_carall_nodes.csv to [placeid]_poi_grid_latlon.csv.

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    if check_extract_zip(PATH["data"] + placeid + "/", placeid + "_carall"):
        for poi_source_here in ["railwaystation", "grid"]:
            with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '_nnidscarall.csv', 'r') as fin, open(PATH["exports_json"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '_latlon.csv', 'w') as fout:
            
                fdata = np.genfromtxt(PATH["data"] + placeid + "/" + placeid + '_carall_nodes.csv', delimiter=',', usecols=(0,1,2))
                for line in fin:
                    poiid = int(line.strip())
                    fdata_lineid = np.argwhere(fdata[:, 2] == poiid)
                    fout.write(str(fdata[fdata_lineid, 0].flatten()[0]) + "," + str(fdata[fdata_lineid, 1].flatten()[0]) + '\n')

        os.remove(PATH["data"] + placeid + "/" + placeid + "_carall_nodes.csv")
        os.remove(PATH["data"] + placeid + "/" + placeid + "_carall_edges.csv")

## GeoJSON Linter

https://geojsonlint.com/

## Copy selected videos, plots, and results

In [ ]:
copyfiles = {"plots": 
                ["_analysis_poi_grid_betweenness.png",
                 "_analysis_poi_grid_closeness.png",
                 "_analysis_poi_grid_random.png",
                 "_analysis_poi_railwaystation_betweenness.png",
                 "_analysis_poi_railwaystation_closeness.png",
                 "_analysis_poi_railwaystation_random.png"
                ],
             "plots_networks":
                ["_biketrack.pdf",
                 "_bikeable.pdf",
                 "_biketrackcarall.pdf",
                 "_carall.pdf",
                 "_carall_poi_grid.pdf",
                 "_carall_poi_railwaystation.pdf",
                 "_MSTabstract_poi_grid.pdf",
                 "_MSTabstract_poi_railwaystation.pdf",
                 "_MSTall_poi_grid.pdf",
                 "_MSTall_poi_railwaystation.pdf",
                 "_GTallcover_poi_grid_Bq1.000.png",
                 "_GTallcover_poi_railwaystation_Bq1.000.png"
                ],
            "videos":
                ["_GTabstract_poi_grid_Bq.mp4",
                 "_GTabstract_poi_grid_Cq.mp4",
                 "_GTabstract_poi_grid_Rq.mp4",
                 "_GTabstract_poi_railwaystation_Bq.mp4",
                 "_GTabstract_poi_railwaystation_Cq.mp4",
                 "_GTabstract_poi_railwaystation_Rq.mp4",
                 "_GTall_poi_grid_Bq.mp4",
                 "_GTall_poi_grid_Cq.mp4",
                 "_GTall_poi_grid_Rq.mp4",
                 "_GTall_poi_railwaystation_Bq.mp4",
                 "_GTall_poi_railwaystation_Cq.mp4",
                 "_GTall_poi_railwaystation_Rq.mp4",
                 "_GTallcover_poi_grid_Bq.mp4",
                 "_GTallcover_poi_grid_Cq.mp4",
                 "_GTallcover_poi_grid_Rq.mp4",
                 "_GTallcover_poi_railwaystation_Bq.mp4",
                 "_GTallcover_poi_railwaystation_Cq.mp4",
                 "_GTallcover_poi_railwaystation_Rq.mp4",
                 "_GTabstract_poi_grid_Bq.webm",
                 "_GTabstract_poi_grid_Cq.webm",
                 "_GTabstract_poi_grid_Rq.webm",
                 "_GTabstract_poi_railwaystation_Bq.webm",
                 "_GTabstract_poi_railwaystation_Cq.webm",
                 "_GTabstract_poi_railwaystation_Rq.webm",
                 "_GTall_poi_grid_Bq.webm",
                 "_GTall_poi_grid_Cq.webm",
                 "_GTall_poi_grid_Rq.webm",
                 "_GTall_poi_railwaystation_Bq.webm",
                 "_GTall_poi_railwaystation_Cq.webm",
                 "_GTall_poi_railwaystation_Rq.webm",
                 "_GTallcover_poi_grid_Bq.webm",
                 "_GTallcover_poi_grid_Cq.webm",
                 "_GTallcover_poi_grid_Rq.webm",
                 "_GTallcover_poi_railwaystation_Bq.webm",
                 "_GTallcover_poi_railwaystation_Cq.webm",
                 "_GTallcover_poi_railwaystation_Rq.webm"
                  ],
             "results":
                ["_existing.csv",
                 "_poi_railwaystation_random.csv",
                 "_poi_grid_random.csv",
                 "_poi_railwaystation_closeness.csv",
                 "_poi_grid_closeness.csv",
                 "_poi_railwaystation_betweenness.csv",
                 "_poi_grid_betweenness.csv"
                ]
            }

In [ ]:
exportpath = PATH["exports_json"] #"/Users/misz/Dropbox/supervision/2021ceciliamorten_bikeviz/data/" 
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    for k, v in copyfiles.items():
        for filename in v:
            try:
                shutil.copy2(PATH[k] + placeid + "/" + placeid + filename, exportpath + placeid + "/")
            except:
                print("File not found: " + placeid + filename)